## Team 2: Michael Kolor and Nikita Roy

In [8]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.api import OLS
import seaborn as sns
from sklearn.linear_model import LogisticRegression

%matplotlib inline

pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 500)

# Read in rankings data
rankings = pd.read_csv('fifa_ranking.csv', encoding = "ISO-8859-1")

# Choose selected columns
rankings = rankings.loc[:,['rank', 'country_full', 'country_abrv', 'confederation', 'rank_date']]

# Clean the data
rankings = rankings.replace({"IR Iran": "Iran"})

rankings['rank_date'] = pd.to_datetime(rankings['rank_date'])

# Select matches after 2015
rankings = rankings[(rankings['rank_date'] > '2015-01-01') & (rankings['rank_date'] < '2018-06-13')]

rankings.shape

# Read in international results data
matches = pd.read_csv('international_results.csv', encoding = "ISO-8859-1")

# Clean the data
matches =  matches.replace({'Germany DR': 'Germany', 'China': 'China PR'})

# Convert data to data time object
matches['date'] = pd.to_datetime(matches['date'])

# Select matches after 2015
matches = matches[(matches['date'] > '2015-01-01') & (matches['date'] < '2018-06-13')]
matches.head()



In [11]:
matches = matches.rename(columns = {"home_team": "team1", "away_team":"team2", "home_score":"score_team1", "away_score":"score_team2"})

matches.head()


,date,team1,team2,score_team1,score_team2,tournament,city,country,neutral
36452,2015-01-04,Bahrain,Jordan,1,0,Friendly,Ballarat,Australia,True
36453,2015-01-04,Iran,Iraq,1,0,Friendly,Wollongong,Australia,True
36454,2015-01-04,Korea Republic,Saudi Arabia,2,0,Friendly,Parramatta,Australia,True
36455,2015-01-04,South Africa,Zambia,1,0,Friendly,Johannesburg,South Africa,False
36456,2015-01-05,China PR,Oman,4,1,Friendly,Penrith,Australia,True


In [12]:
# Get rankings for every day
rankings = rankings.set_index(['rank_date'])\
            .groupby(['country_full'], group_keys=False)\
            .resample('D').first()\
            .fillna(method='ffill')\
            .reset_index()

# join the ranks and international results data set
df = matches.merge(rankings, 
                        left_on=['date', 'team1'], 
                        right_on=['rank_date', 'country_full'])
df = df.merge(rankings, 
                        left_on=['date', 'team2'], 
                        right_on=['rank_date', 'country_full'], 
                        suffixes=('_team1', '_team2'))

In [13]:
df.columns

Index(['date', 'team1', 'team2', 'score_team1', 'score_team2', 'tournament', 'city', 'country', 'neutral', 'rank_date_team1', 'rank_team1', 'country_full_team1', 'country_abrv_team1', 'confederation_team1', 'rank_date_team2', 'rank_team2', 'country_full_team2', 'country_abrv_team2', 'confederation_team2'], dtype='object')

In [14]:
# Get winning team for each match
winner = []
for i in range(len(df['team1'])):
    if df['score_team1'][i] > df['score_team2'][i]:
        winner.append(df['team1'][i])
    elif df['score_team1'][i] < df ['score_team2'][i]:
        winner.append(df['team2'][i])
    else:
        winner.append('Draw')
df['winning_team'] = winner
df['goal_difference'] = np.absolute(df['score_team1'] - df['score_team2'])

df = df.drop(['city', 'country', 'rank_date_team1', 'rank_date_team2', 'country_full_team1', 'country_full_team2'], axis = 1)
df.head()



,date,team1,team2,score_team1,score_team2,tournament,neutral,rank_team1,country_abrv_team1,confederation_team1,rank_team2,country_abrv_team2,confederation_team2,winning_team,goal_difference
0,2015-01-09,Australia,Kuwait,4,1,AFC Asian Cup,False,100.0,AUS,AFC,125.0,KUW,AFC,Australia,3
1,2015-01-09,Senegal,Gabon,1,0,Friendly,True,35.0,SEN,CAF,62.0,GAB,CAF,Senegal,1
2,2015-01-10,Burkina Faso,Swaziland,5,1,Friendly,True,64.0,BFA,CAF,164.0,SWZ,CAF,Burkina Faso,4
3,2015-01-10,Cameroon,South Africa,1,1,Friendly,True,42.0,CMR,CAF,52.0,RSA,CAF,Draw,0
4,2015-01-10,Korea Republic,Oman,1,0,AFC Asian Cup,True,69.0,KOR,AFC,93.0,OMA,AFC,Korea Republic,1


In [15]:
# Create Year column to be added to data frame
year = []
for date in df['date']:
    year.append(date.year)

df['year'] = year

# Assign 2018 as 2017, since we will be using 2017 data that is available for variable values

df['year'] = df['year'].replace(2018,2017)

In [16]:
df.head()

,date,team1,team2,score_team1,score_team2,tournament,neutral,rank_team1,country_abrv_team1,confederation_team1,rank_team2,country_abrv_team2,confederation_team2,winning_team,goal_difference,year
0,2015-01-09,Australia,Kuwait,4,1,AFC Asian Cup,False,100.0,AUS,AFC,125.0,KUW,AFC,Australia,3,2015
1,2015-01-09,Senegal,Gabon,1,0,Friendly,True,35.0,SEN,CAF,62.0,GAB,CAF,Senegal,1,2015
2,2015-01-10,Burkina Faso,Swaziland,5,1,Friendly,True,64.0,BFA,CAF,164.0,SWZ,CAF,Burkina Faso,4,2015
3,2015-01-10,Cameroon,South Africa,1,1,Friendly,True,42.0,CMR,CAF,52.0,RSA,CAF,Draw,0,2015
4,2015-01-10,Korea Republic,Oman,1,0,AFC Asian Cup,True,69.0,KOR,AFC,93.0,OMA,AFC,Korea Republic,1,2015


In [17]:
# Read in GDP data
gdp = pd.read_csv('gdp_data.csv', encoding = "ISO-8859-1")

# Only select columns from 2015 onwards
gdp = gdp.drop(gdp.columns[[range(5,59)]], axis = 1)
gdp.head()
gdp.columns

# Transform data from wide format to long format
gdp = pd.melt(gdp, id_vars = ['Country Name', 'Country Code'], value_vars = [ '2015', '2016', '2017'])

gdp.columns = ['Country', 'Country Code', 'Year', 'GDP']
gdp = gdp.fillna(0)
gdp.head()
gdp['Year'] = gdp['Year'].astype('int64')

gdp.head()

,Country,Country Code,Year,GDP
0,Aruba,ABW,2015,0.000000e+00
1,Afghanistan,AFG,2015,1.921556e+10
2,Angola,AGO,2015,1.026210e+11
3,Albania,ALB,2015,1.138693e+10
4,Andorra,AND,2015,2.811489e+09


In [18]:
# Read in population data
population = pd.read_csv('population_data.csv', encoding = "ISO-8859-1")

# Only select columns from 2015 onwards
population = population.drop(population.columns[[range(4,59)]], axis = 1)
population.head()
population.columns

# Transform data from wide format to long format
population = pd.melt(population, id_vars = ['Country Name'], value_vars = ['2015', '2016', '2017'])

population.columns = ['Country',  'Year', 'population']
population = population.fillna(0)
population.head()
population['Year'] = population['Year'].astype('int64')

population.head()



,Country,Year,population
0,Aruba,2015,104341.0
1,Afghanistan,2015,33736494.0
2,Angola,2015,27859305.0
3,Albania,2015,2880703.0
4,Andorra,2015,78014.0


In [19]:
# Merge GDP and population data
gdp = gdp.merge(population, left_on=['Country', 'Year'], right_on=['Country', 'Year']) 
gdp.head()

,Country,Country Code,Year,GDP,population
0,Aruba,ABW,2015,0.000000e+00,104341.0
1,Afghanistan,AFG,2015,1.921556e+10,33736494.0
2,Angola,AGO,2015,1.026210e+11,27859305.0
3,Albania,ALB,2015,1.138693e+10,2880703.0
4,Andorra,AND,2015,2.811489e+09,78014.0


In [20]:
# Calculate GDP per capita
gdp['GDP_per_capita'] = gdp['GDP']/gdp['population']
gdp.head()


,Country,Country Code,Year,GDP,population,GDP_per_capita
0,Aruba,ABW,2015,0.000000e+00,104341.0,0.000000
1,Afghanistan,AFG,2015,1.921556e+10,33736494.0,569.577923
2,Angola,AGO,2015,1.026210e+11,27859305.0,3683.544869
3,Albania,ALB,2015,1.138693e+10,2880703.0,3952.830781
4,Andorra,AND,2015,2.811489e+09,78014.0,36038.267606


In [21]:
df_copy = df.copy()

In [22]:
# Merge GDP and Dataframe
df = df.merge(gdp, 
                left_on=['year', 'team1'], 
                right_on=['Year', 'Country'])


df = df.merge(gdp, 
                left_on=['year', 'team2'], 
                right_on=['Year', 'Country'], 
                suffixes=('_team1', '_team2'))


In [23]:
df = df.drop(['Country_team1', 'Country Code_team1', 'Year_team1','GDP_team1', 'Country_team2',
       'Country Code_team2', 'Year_team2', 'GDP_team2'], axis = 1)

In [24]:
#2 points if team1  wins, 1, if it is a draw, 0 if team1 team loses
df.loc[df.winning_team == df.team1, 'winning_team'] = 2
df.loc[df.winning_team == 'Draw', 'winning_team'] = 1
df.loc[df.winning_team == df.team2, 'winning_team'] = 0

In [25]:
df.head()

,date,team1,team2,score_team1,score_team2,tournament,neutral,rank_team1,country_abrv_team1,confederation_team1,rank_team2,country_abrv_team2,confederation_team2,winning_team,goal_difference,year,population_team1,GDP_per_capita_team1,population_team2,GDP_per_capita_team2
0,2015-01-09,Australia,Kuwait,4,1,AFC Asian Cup,False,100.0,AUS,AFC,125.0,KUW,AFC,2,3,2015,23850784.0,56561.243438,3935794.0,29108.993001
1,2015-06-05,Jordan,Kuwait,2,2,Friendly,True,103.0,JOR,AFC,125.0,KUW,AFC,1,0,2015,9159302.0,4096.099275,3935794.0,29108.993001
2,2015-01-17,Oman,Kuwait,1,0,AFC Asian Cup,True,93.0,OMA,AFC,125.0,KUW,AFC,2,1,2015,4199810.0,16406.711618,3935794.0,29108.993001
3,2015-06-11,Lebanon,Kuwait,0,1,FIFA World Cup qualification,False,135.0,LIB,AFC,125.0,KUW,AFC,0,1,2015,5851479.0,8452.443641,3935794.0,29108.993001
4,2015-01-13,Australia,Oman,4,0,AFC Asian Cup,False,100.0,AUS,AFC,93.0,OMA,AFC,2,4,2015,23850784.0,56561.243438,4199810.0,16406.711618


In [26]:
# Read in player data
player_data = pd.read_csv("complete.csv", encoding = "ISO-8859-1")

countries = []
sizes = []

# data for players in top 200
for country, player_df in player_data.sort_values(by="overall", ascending=False).head(200).groupby("nationality"):
    countries.append(country)
    sizes.append(player_df["overall"].size)

top_200_df = pd.DataFrame()
top_200_df["country"] = countries
top_200_df["players_in_top_200"] = sizes 
top_200_df

team_roster_data = pd.DataFrame()
# hOW MNY PLAYERS IN TOP 200

# Get top 23 players
for country, curr_df in player_data.groupby("nationality"):
    #print(country)
    country_df = curr_df.sort_values(by="overall", ascending=False)
    country_df["nationality"] = country
    country_df = country_df.head(23)
    country_df["number_of_players"] = country_df["nationality"].size
    #print(country_df.head(23)["eur_value"].mean()/23)
    team_roster_data = team_roster_data.append(country_df)

# Merge players in top 200
top_200_team_roster_merged_df = pd.merge(team_roster_data, top_200_df, left_on="nationality", right_on="country", how="left")
top_200_team_roster_merged_df[top_200_team_roster_merged_df["nationality"] == "Spain"]
top_200_team_roster_merged_df["players_in_top_200"].fillna(0, inplace=True)
top_200_team_roster_merged_df = top_200_team_roster_merged_df.drop("country", axis = 1)
top_200_team_roster_merged_df["epl"] = top_200_team_roster_merged_df["league"] == "English Premier League"
top_200_team_roster_merged_df["laliga"] = top_200_team_roster_merged_df["league"] == "Spanish Primera División"
top_200_team_roster_merged_df["bundesliga"] = top_200_team_roster_merged_df["league"] == "German Bundesliga"
top_200_team_roster_merged_df[top_200_team_roster_merged_df["nationality"] == "Belgium"]

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,weight_kg,body_type,real_face,flag,nationality,photo,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,work_rate_def,preferred_foot,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,rs,rw,rf,ram,rcm,rm,rdm,rcb,rb,rwb,st,lw,cf,cam,cm,lm,cdm,cb,lb,lwb,ls,lf,lam,lcm,ldm,lcb,gk,1_on_1_rush_trait,acrobatic_clearance_trait,argues_with_officials_trait,avoids_using_weaker_foot_trait,backs_into_player_trait,bicycle_kicks_trait,cautious_with_crosses_trait,chip_shot_trait,chipped_penalty_trait,comes_for_crosses_trait,corner_specialist_trait,diver_trait,dives_into_tackles_trait,diving_header_trait,driven_pass_trait,early_crosser_trait,fan's_favourite_trait,fancy_flicks_trait,finesse_shot_trait,flair_trait,flair_passes_trait,gk_flat_kick_trait,gk_long_throw_trait,gk_up_for_corners_trait,giant_throw_in_trait,inflexible_trait,injury_free_trait,injury_prone_trait,leadership_trait,long_passer_trait,long_shot_taker_trait,long_throw_in_trait,one_club_player_trait,outside_foot_shot_trait,playmaker_trait,power_free_kick_trait,power_header_trait,puncher_trait,rushes_out_of_goal_trait,saves_with_feet_trait,second_wind_trait,selfish_trait,skilled_dribbling_trait,stutter_penalty_trait,swerve_pass_trait,takes_finesse_free_kicks_trait,target_forward_trait,team_player_trait,technical_dribbler_trait,tries_to_beat_defensive_line_trait,poacher_speciality,speedster_speciality,aerial_threat_speciality,dribbler_speciality,playmaker_speciality,engine_speciality,distance_shooter_speciality,crosser_speciality,free_kick_specialist_speciality,tackling_speciality,tactician_speciality,acrobat_speciality,strength_speciality,clinical_finisher_speciality,prefers_rs,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk,number_of_players,players_in_top_200,epl,laliga,bundesliga
168,183277,E. Hazard,Eden Hazard,Chelsea,https://cdn.sofifa.org/18/teams/5.png,2096,26,English Premier League,1991-01-07,173,76,Normal,True,https://cdn.sofifa.org/flags/7@3x.png,Belgium,https://cdn.sofifa.org/18/players/183277.png,90500000,295000,174200000.0,90,91,90,82,84,92,32,66,4,4,4,High,Medium,Right,80,83,57,86,79,93,82,79,81,92,93,87,93,85,91,79,59,79,65,82,54,41,85,86,86,87,25,27,22,11,12,6,8,8,82.0,88.0,87.0,88.0,81.0,87.0,61.0,47.0,59.0,64.0,82.0,88.0,87.0,88.0,81.0,87.0,61.0,47.0,59.0,64.0,82.0,87.0,88.0,81.0,61.0,47.0,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,23,11.0,True,False,False
169,192119,T. Courtois,Thibaut Courtois,Chelsea,https://cdn.sofifa.org/18/teams/5.png,1282,25,English Premier League,1992-05-11,199,94,Courtois,True,https://cdn.sofifa.org/flags/7@3x.png,Belgium,https://cdn.sofifa.org/18/players/192119.png,59000000,190000,113600000.0,89,92,85,91,69,88,49,86,4,1,3,Medium,Medium,Left,14,14,13,32,12,13,19,11,31,23,46,52,61,81,45,36,68,38,70,17,23,15,13,44,27,52,

In [27]:
country_player_data = top_200_team_roster_merged_df.groupby("nationality").agg({
    "laliga": np.sum,
    "epl": np.sum,
    "bundesliga": np.sum,
    "players_in_top_200": np.median,
    "number_of_players": np.median,
    "age": np.mean,
    "overall": np.sum,
    "eur_value": np.sum,  
})

In [28]:
country_player_data.head()



,laliga,epl,bundesliga,players_in_top_200,number_of_players,age,overall,eur_value
nationality,,,,,,,,
Afghanistan,False,1.0,0.0,0.0,3,22.000000,169,690000
Albania,False,0.0,2.0,0.0,23,26.130435,1633,74975000
Algeria,False,2.0,2.0,1.0,23,26.782609,1749,229700000
Angola,False,0.0,0.0,0.0,15,25.600000,1024,37095000
Antigua & Barbuda,False,0.0,0.0,0.0,4,23.000000,238,1410000


In [29]:
# mean imputate 50 for players not listed in dataset
country_player_data["overall_average"] = (country_player_data["overall"] + (23-country_player_data["number_of_players"]) * 50)/23
country_player_data["top_3_leagues"] = country_player_data["laliga"] + country_player_data["epl"] + country_player_data["bundesliga"]

# drop overall and number of players
country_player_data_cleaned = country_player_data.copy().drop(["overall","number_of_players"], axis = 1)


In [30]:
country_player_data_cleaned = country_player_data_cleaned.drop(['laliga', 'epl', 'bundesliga'], axis = 1)


In [31]:
country_player_data_cleaned['nationality'] = country_player_data_cleaned.index

In [32]:
# Merge Player Data and Dataframe
df = df.merge(country_player_data_cleaned, 
                left_on='team1', 
                right_on= 'nationality', how = 'left')


df = df.merge(country_player_data_cleaned, 
                left_on= 'team2', 
                right_on= 'nationality', how = 'left',
                suffixes=('_team1', '_team2'))



In [33]:
df.head()

,date,team1,team2,score_team1,score_team2,tournament,neutral,rank_team1,country_abrv_team1,confederation_team1,rank_team2,country_abrv_team2,confederation_team2,winning_team,goal_difference,year,population_team1,GDP_per_capita_team1,population_team2,GDP_per_capita_team2,players_in_top_200_team1,age_team1,eur_value_team1,overall_average_team1,top_3_leagues_team1,nationality_team1,players_in_top_200_team2,age_team2,eur_value_team2,overall_average_team2,top_3_leagues_team2,nationality_team2
0,2015-01-09,Australia,Kuwait,4,1,AFC Asian Cup,False,100.0,AUS,AFC,125.0,KUW,AFC,2,3,2015,23850784.0,56561.243438,3935794.0,29108.993001,0.0,27.565217,85900000.0,72.869565,4.0,Australia,0.0,27.5,2070000.0,51.391304,0.0,Kuwait
1,2015-06-05,Jordan,Kuwait,2,2,Friendly,True,103.0,JOR,AFC,125.0,KUW,AFC,1,0,2015,9159302.0,4096.099275,3935794.0,29108.993001,NaN,NaN,NaN,NaN,NaN,NaN,0.0,27.5,2070000.0,51.391304,0.0,Kuwait
2,2015-01-17,Oman,Kuwait,1,0,AFC Asian Cup,True,93.0,OMA,AFC,125.0,KUW,AFC,2,1,2015,4199810.0,16406.711618,3935794.0,29108.993001,0.0,35.000000,950000.0,51.000000,0.0,Oman,0.0,27.5,2070000.0,51.391304,0.0,Kuwait
3,2015-06-11,Lebanon,Kuwait,0,1,FIFA World Cup qualification,False,135.0,LIB,AFC,125.0,KUW,AFC,0,1,2015,5851479.0,8452.443641,3935794.0,29108.993001,0.0,22.800000,3020000.0,52.739130,0.0,Lebanon,0.0,27.5,2070000.0,51.391304,0.0,Kuwait
4,2015-01-13,Australia,Oman,4,0,AFC Asian Cup,False,100.0,AUS,AFC,93.0,OMA,AFC,2,4,2015,23850784.0,56561.243438,4199810.0,16406.711618,0.0,27.565217,85900000.0,72.869565,4.0,Australia,0.0,35.0,950000.0,51.000000,0.0,Oman


In [34]:
df.shape

(2017, 32)

In [35]:
df.columns

Index(['date', 'team1', 'team2', 'score_team1', 'score_team2', 'tournament', 'neutral', 'rank_team1', 'country_abrv_team1', 'confederation_team1', 'rank_team2', 'country_abrv_team2', 'confederation_team2', 'winning_team', 'goal_difference', 'year', 'population_team1', 'GDP_per_capita_team1', 'population_team2', 'GDP_per_capita_team2', 'players_in_top_200_team1', 'age_team1', 'eur_value_team1', 'overall_average_team1', 'top_3_leagues_team1', 'nationality_team1', 'players_in_top_200_team2', 'age_team2', 'eur_value_team2', 'overall_average_team2', 'top_3_leagues_team2', 'nationality_team2'], dtype='object')

In [36]:
df = pd.get_dummies(df, columns=['confederation_team1', 'confederation_team2'], drop_first=True)

df.head()

,date,team1,team2,score_team1,score_team2,tournament,neutral,rank_team1,country_abrv_team1,rank_team2,country_abrv_team2,winning_team,goal_difference,year,population_team1,GDP_per_capita_team1,population_team2,GDP_per_capita_team2,players_in_top_200_team1,age_team1,eur_value_team1,overall_average_team1,top_3_leagues_team1,nationality_team1,players_in_top_200_team2,age_team2,eur_value_team2,overall_average_team2,top_3_leagues_team2,nationality_team2,confederation_team1_CAF,confederation_team1_CONCACAF,confederation_team1_CONMEBOL,confederation_team1_OFC,confederation_team1_UEFA,confederation_team2_CAF,confederation_team2_CONCACAF,confederation_team2_CONMEBOL,confederation_team2_OFC,confederation_team2_UEFA
0,2015-01-09,Australia,Kuwait,4,1,AFC Asian Cup,False,100.0,AUS,125.0,KUW,2,3,2015,23850784.0,56561.243438,3935794.0,29108.993001,0.0,27.565217,85900000.0,72.869565,4.0,Australia,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0
1,2015-06-05,Jordan,Kuwait,2,2,Friendly,True,103.0,JOR,125.0,KUW,1,0,2015,9159302.0,4096.099275,3935794.0,29108.993001,NaN,NaN,NaN,NaN,NaN,NaN,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0
2,2015-01-17,Oman,Kuwait,1,0,AFC Asian Cup,True,93.0,OMA,125.0,KUW,2,1,2015,4199810.0,16406.711618,3935794.0,29108.993001,0.0,35.000000,950000.0,51.000000,0.0,Oman,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0
3,2015-06-11,Lebanon,Kuwait,0,1,FIFA World Cup qualification,False,135.0,LIB,125.0,KUW,0,1,2015,5851479.0,8452.443641,3935794.0,29108.993001,0.0,22.800000,3020000.0,52.739130,0.0,Lebanon,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0
4,2015-01-13,Australia,Oman,4,0,AFC Asian Cup,False,100.0,AUS,93.0,OMA,2,4,2015,23850784.0,56561.243438,4199810.0,16406.711618,0.0,27.565217,85900000.0,72.869565,4.0,Australia,0.0,35.0,950000.0,51.000000,0.0,Oman,0,0,0,0,0,0,0,0,0,0


In [37]:
host = []

for i in range(len(df['team1'])):
    host.append(int(df.loc[i,['neutral']].values*1))



In [38]:
df['host_team1'] = host

In [39]:
df = df.drop(['neutral', 'tournament', 'country_abrv_team1', 'country_abrv_team2', 'date', 'year'], axis = 1)

In [40]:
df.columns

Index(['team1', 'team2', 'score_team1', 'score_team2', 'rank_team1', 'rank_team2', 'winning_team', 'goal_difference', 'population_team1', 'GDP_per_capita_team1', 'population_team2', 'GDP_per_capita_team2', 'players_in_top_200_team1', 'age_team1', 'eur_value_team1', 'overall_average_team1', 'top_3_leagues_team1', 'nationality_team1', 'players_in_top_200_team2', 'age_team2', 'eur_value_team2', 'overall_average_team2', 'top_3_leagues_team2', 'nationality_team2', 'confederation_team1_CAF', 'confederation_team1_CONCACAF', 'confederation_team1_CONMEBOL', 'confederation_team1_OFC', 'confederation_team1_UEFA', 'confederation_team2_CAF', 'confederation_team2_CONCACAF', 'confederation_team2_CONMEBOL', 'confederation_team2_OFC', 'confederation_team2_UEFA', 'host_team1'], dtype='object')

In [41]:
df['rank_diff'] = df['rank_team1'] - df['rank_team2']

In [42]:
df.head()

,team1,team2,score_team1,score_team2,rank_team1,rank_team2,winning_team,goal_difference,population_team1,GDP_per_capita_team1,population_team2,GDP_per_capita_team2,players_in_top_200_team1,age_team1,eur_value_team1,overall_average_team1,top_3_leagues_team1,nationality_team1,players_in_top_200_team2,age_team2,eur_value_team2,overall_average_team2,top_3_leagues_team2,nationality_team2,confederation_team1_CAF,confederation_team1_CONCACAF,confederation_team1_CONMEBOL,confederation_team1_OFC,confederation_team1_UEFA,confederation_team2_CAF,confederation_team2_CONCACAF,confederation_team2_CONMEBOL,confederation_team2_OFC,confederation_team2_UEFA,host_team1,rank_diff
0,Australia,Kuwait,4,1,100.0,125.0,2,3,23850784.0,56561.243438,3935794.0,29108.993001,0.0,27.565217,85900000.0,72.869565,4.0,Australia,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0,0,-25.0
1,Jordan,Kuwait,2,2,103.0,125.0,1,0,9159302.0,4096.099275,3935794.0,29108.993001,NaN,NaN,NaN,NaN,NaN,NaN,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0,1,-22.0
2,Oman,Kuwait,1,0,93.0,125.0,2,1,4199810.0,16406.711618,3935794.0,29108.993001,0.0,35.000000,950000.0,51.000000,0.0,Oman,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0,1,-32.0
3,Lebanon,Kuwait,0,1,135.0,125.0,0,1,5851479.0,8452.443641,3935794.0,29108.993001,0.0,22.800000,3020000.0,52.739130,0.0,Lebanon,0.0,27.5,2070000.0,51.391304,0.0,Kuwait,0,0,0,0,0,0,0,0,0,0,0,10.0
4,Australia,Oman,4,0,100.0,93.0,2,4,23850784.0,56561.243438,4199810.0,16406.711618,0.0,27.565217,85900000.0,72.869565,4.0,Australia,0.0,35.0,950000.0,51.000000,0.0,Oman,0,0,0,0,0,0,0,0,0,0,0,7.0


In [43]:
df = df.fillna(0)

In [44]:
df.to_csv('worldcup_df.csv')

In [45]:
df.shape

(2017, 36)

## Creating Prediction Set

In [46]:
world_cup = pd.read_csv('world_cup_2018.csv', encoding = "ISO-8859-1")


world_cup.head()

,Team,Group,Previous appearances appearances,Previous titles titles,Previous finals,Previous semifinals,Current FIFA rank,First match against,Match index,history with first opponent W-L,history with first opponent goals,Second match against,Match index.1,history with second opponent W-L,history with second opponent goals,Third match against,Match index.2,history with third opponent W-L,history with third opponent goals
0,Russia,A,10,0,0,1,65,Saudi Arabia,1,-1.0,-2.0,Egypt,17,NaN,NaN,Uruguay,33,0.0,0.0
1,Saudi Arabia,A,4,0,0,0,63,Russia,1,1.0,2.0,Uruguay,18,1.0,1.0,Egypt,34,-5.0,-5.0
2,Egypt,A,2,0,0,0,31,Uruguay,2,-1.0,-2.0,Russia,17,NaN,NaN,Saudi Arabia,34,5.0,5.0
3,Uruguay,A,12,2,2,5,21,Egypt,2,1.0,2.0,Saudi Arabia,18,-1.0,-1.0,Russia,33,0.0,0.0
4,Portugal,B,6,0,0,2,3,Spain,3,-12.0,-31.0,Morocco,19,-1.0,-2.0,Iran,35,2.0,5.0


In [47]:
# Read in World Cup 2018 data
world_cup = pd.read_csv('world_cup_2018.csv', encoding = "ISO-8859-1")
world_cup = world_cup[['Team', 'Group', 'First match \ragainst', 'Second match\r against', 'Third match\r against']]

# Clean Data
world_cup = world_cup.dropna(how='all')
world_cup = world_cup.replace({"IRAN": "Iran", 
                               "Costarica": "Costa Rica", 
                               "Porugal": "Portugal", 
                               "Columbia": "Colombia", 
                                })
world_cup = world_cup.set_index('Team')
world_cup['Team'] = world_cup.index

# Reshape data frame
world_cup = pd.melt(world_cup, id_vars = ['Team', 'Group'], value_vars = ['First match \ragainst', 'Second match\r against',
       'Third match\r against'])
world_cup.columns
world_cup.head()




,Team,Group,variable,value
0,Russia,A,First match \ragainst,Saudi Arabia
1,Saudi Arabia,A,First match \ragainst,Russia
2,Egypt,A,First match \ragainst,Uruguay
3,Uruguay,A,First match \ragainst,Egypt
4,Portugal,B,First match \ragainst,Spain


In [48]:
team1 = []
for i in range(len(world_cup['Team'])):
    if world_cup.loc[i, 'Team'] > world_cup.loc[i, 'value']:
        team1.append(world_cup.loc[i, 'Team'])
    else:
        team1.append(world_cup.loc[i, 'value'])
world_cup['team1'] = team1

team2 = []
for i in range(len(world_cup['value'])):
    if world_cup.loc[i, 'Team'] < world_cup.loc[i, 'value']:
        team2.append(world_cup.loc[i, 'Team'])
    else:
        team2.append(world_cup.loc[i, 'value'])
world_cup['team2'] = team2

world_cup = world_cup[['team1', 'team2', 'Group']]

In [49]:
world_cup = world_cup.drop_duplicates(subset = ['team1', 'team2'], keep = 'first')
world_cup.head()

,team1,team2,Group
0,Saudi Arabia,Russia,A
2,Uruguay,Egypt,A
4,Spain,Portugal,B
6,Morocco,Iran,B
8,France,Australia,C


In [50]:
world_cup = world_cup.reset_index(drop = True)

In [51]:
world_cup

,team1,team2,Group
0,Saudi Arabia,Russia,A
1,Uruguay,Egypt,A
2,Spain,Portugal,B
3,Morocco,Iran,B
4,France,Australia,C
5,Peru,Denmark,C
6,Iceland,Argentina,D
7,Nigeria,Croatia,D
8,Switzerland,Brazil,E
9,Serbia,Costa Rica,E


In [52]:
world_cup.shape

(49, 3)

In [53]:
rankings.head()

,rank_date,rank,country_full,country_abrv,confederation
0,2015-01-08,142.0,Afghanistan,AFG,AFC
1,2015-01-09,142.0,Afghanistan,AFG,AFC
2,2015-01-10,142.0,Afghanistan,AFG,AFC
3,2015-01-11,142.0,Afghanistan,AFG,AFC
4,2015-01-12,142.0,Afghanistan,AFG,AFC


In [54]:
world_cup.head()

,team1,team2,Group
0,Saudi Arabia,Russia,A
1,Uruguay,Egypt,A
2,Spain,Portugal,B
3,Morocco,Iran,B
4,France,Australia,C


In [55]:
rankings['rank_date'] = pd.to_datetime(rankings['rank_date'])
rankings = rankings.loc[rankings['rank_date'] == '2018-06-07']
rankings.head()

world_cup = world_cup.merge(rankings, left_on = 'team1', right_on = 'country_full', how = 'left')
world_cup.head()


world_cup = world_cup.merge(rankings, left_on = 'team2', right_on = 'country_full', how = 'left', suffixes = ['_team1', '_team2'])
world_cup = world_cup[['team1', 'team2', 'rank_team1','rank_team2', 'confederation_team1', 'confederation_team2', 'Group']]


world_cup.shape


(49, 7)

In [56]:
# world_cup.rename(columns = {'rank' : 'Rank2'})
world_cup['rank_diff'] = world_cup['rank_team1'] - world_cup['rank_team2']


In [57]:
# Create list to record if host country is playing match
host = []

for i in range(len(world_cup['team1'])):
    if world_cup[['team1']].values[i] == 'Russia':
        host.append(1)
    elif world_cup[['team2']].values[i] == 'Russia':
        host.append(1)
    else: 
        host.append(0)
        
world_cup['host_team1'] = host

In [58]:
world_cup.head()

,team1,team2,rank_team1,rank_team2,confederation_team1,confederation_team2,Group,rank_diff,host_team1
0,Saudi Arabia,Russia,67.0,70.0,AFC,UEFA,A,-3.0,1
1,Uruguay,Egypt,14.0,45.0,CONMEBOL,CAF,A,-31.0,0
2,Spain,Portugal,10.0,4.0,UEFA,UEFA,B,6.0,0
3,Morocco,Iran,41.0,37.0,CAF,AFC,B,4.0,0
4,France,Australia,7.0,36.0,UEFA,AFC,C,-29.0,0


In [59]:
world_cup.shape

(49, 9)

In [60]:
gdp = gdp[(gdp['Year']) == 2017]


In [61]:
gdp.head()

,Country,Country Code,Year,GDP,population,GDP_per_capita
506,Aruba,ABW,2017,0.000000e+00,105264.0,0.000000
507,Afghanistan,AFG,2017,2.081530e+10,35530081.0,585.850064
508,Angola,AGO,2017,1.242090e+11,29784193.0,4170.299326
509,Albania,ALB,2017,1.303935e+10,2873457.0,4537.862492
510,Andorra,AND,2017,3.012914e+09,76965.0,39146.548834


In [62]:
world_cup["team1"].values

array(['Saudi Arabia', 'Uruguay', 'Spain', 'Morocco', 'France', 'Peru',
       'Iceland', 'Nigeria', 'Switzerland', 'Serbia', 'Mexico', 'Sweden',
       'Sweden', 'Panama', 'Tunisia', 'Senegal', 'Japan', 'Russia',
       'Uruguay', 'Portugal', 'Spain', 'Peru', 'Denmark', 'Croatia',
       'Nigeria', 'Costa Rica', 'Switzerland', 'Sweden', 'Mexico',
       'Tunisia', 'Panama', 'Poland', 'Senegal', 'Uruguay', 'Saudi Arabia',
       'Portugal', 'Spain', 'France', 'Peru', 'Nigeria', 'Iceland',
       'Serbia', 'Switzerland', 'Korea Republic', 'Sweden', 'England',
       'Tunisia', 'Poland', 'Senegal'], dtype=object)

In [63]:
# Merge GDP and Dataframe
world_cup = world_cup.merge(gdp, 
                left_on=['team1'], 
                right_on=['Country'], how = 'left')


world_cup = world_cup.merge(gdp, 
                left_on=['team2'], 
                right_on=['Country'], how = 'left',
                suffixes=('_team1', '_team2'))


In [64]:
world_cup.shape

(49, 21)

In [65]:
# Merge Player Data and Dataframe
world_cup = world_cup.merge(country_player_data_cleaned, 
                left_on='team1', 
                right_on= 'nationality', how ='left')


world_cup = world_cup.merge(country_player_data_cleaned, 
                left_on= 'team2', 
                right_on= 'nationality', how = 'left',
                suffixes=('_team1', '_team2'))

In [66]:
world_cup.shape

(49, 33)

In [67]:
world_cup = pd.get_dummies(world_cup, columns=['confederation_team1', 'confederation_team2'], drop_first=True)



In [68]:
#Creating columns for confederations
world_cup['confederation_team1_CAF'] = 0
world_cup['confederation_team1_OFC'] = 0
world_cup['confederation_team2_OFC'] = 0

In [69]:
world_cup = world_cup[['team1', 'team2','rank_team1',
       'rank_team2', 'population_team1',
       'GDP_per_capita_team1', 'population_team2', 'GDP_per_capita_team2',
       'players_in_top_200_team1', 'age_team1', 'eur_value_team1',
       'overall_average_team1', 'top_3_leagues_team1', 
       'players_in_top_200_team2', 'age_team2', 'eur_value_team2',
       'overall_average_team2', 'top_3_leagues_team2', 
       'confederation_team1_CAF', 'confederation_team1_CONCACAF',
       'confederation_team1_CONMEBOL', 'confederation_team1_OFC',
       'confederation_team1_UEFA', 'confederation_team2_CAF',
       'confederation_team2_CONCACAF', 'confederation_team2_CONMEBOL',
       'confederation_team2_OFC', 'confederation_team2_UEFA', 'host_team1',
       'rank_diff', 'Group']]

In [70]:
world_cup.shape

(49, 31)

In [71]:
world_cup.to_csv('worldcup_2018_prediction_df.csv')

In [72]:
world_cup.shape

(49, 31)

In [73]:
world_cup

,team1,team2,rank_team1,rank_team2,population_team1,GDP_per_capita_team1,population_team2,GDP_per_capita_team2,players_in_top_200_team1,age_team1,eur_value_team1,overall_average_team1,top_3_leagues_team1,players_in_top_200_team2,age_team2,eur_value_team2,overall_average_team2,top_3_leagues_team2,confederation_team1_CAF,confederation_team1_CONCACAF,confederation_team1_CONMEBOL,confederation_team1_OFC,confederation_team1_UEFA,confederation_team2_CAF,confederation_team2_CONCACAF,confederation_team2_CONMEBOL,confederation_team2_OFC,confederation_team2_UEFA,host_team1,rank_diff,Group
0,Saudi Arabia,Russia,67.0,70.0,32938213.0,20760.901631,144495044.0,10917.467868,0.0,27.304348,85700000,72.478261,0.0,0.0,29.130435,216200000.0,78.478261,0.0,0,0,0,0,0,0,0,0,0,1,1,-3.0,A
1,Uruguay,Egypt,14.0,45.0,3456750.0,16245.598368,97553151.0,2412.725756,4.0,27.130435,400500000,79.217391,0.0,1.0,27.173913,92320000.0,71.652174,4.0,0,0,1,0,0,1,0,0,0,0,0,-31.0,A
2,Spain,Portugal,10.0,4.0,46572028.0,28156.815503,10293718.0,21136.289143,32.0,28.173913,905500000,85.695652,11.0,11.0,27.086957,545000000.0,82.434783,4.0,0,0,0,0,1,0,0,0,0,1,0,6.0,B
3,Morocco,Iran,41.0,37.0,35739580.0,3053.729227,81162788.0,5415.215653,1.0,26.695652,232300000,76.608696,1.0,0.0,25.000000,44780000.0,62.695652,0.0,0,0,0,0,0,0,0,0,0,0,0,4.0,B
4,France,Australia,7.0,36.0,67118648.0,38476.639160,24598933.0,53799.894491,20.0,26.739130,811000000,84.347826,7.0,0.0,27.565217,85900000.0,72.869565,4.0,0,0,0,0,1,0,0,0,0,0,0,-29.0,C
5,Peru,Denmark,11.0,12.0,32165485.0,6571.920181,5769603.0,56307.513706,0.0,26.043478,60700000,70.826087,2.0,2.0,25.652174,309500000.0,77.608696,11.0,0,0,1,0,0,0,0,0,0,1,0,-1.0,C
6,Iceland,Argentina,22.0,5.0,341284.0,70056.873393,44271041.0,14401.965384,0.0,27.478261,73350000,71.173913,3.0,16.0,28.695652,741500000.0,83.782609,4.0,0,0,0,0,1,0,0,1,0,0,0,17.0,D
7,Nigeria,Croatia,48.0,20.0,190886311.0,1968.559181,4125700.0,13294.514926,0.0,24.565217,171900000,74.739130,7.0,6.0,26.304348,431300000.0,79.956522,3.0,0,0,0,0,0,0,0,0,0,1,0,28.0,D
8,Switzerland,Brazil,6.0,2.0,8466017.0,80189.657073,209288278.0,9821.429177,2.0,27.565217,258600000,77.956522,12.0,17.0,27.608696,723500000.0,84.304348,6.0,0,0,0,0,1,0,0,1,0,0,0,4.0,E
9,Serbia,Costa Rica,34.0,23.0,7022268.0,5900.038107,4905769.0,11630.668396,1.0,27.043478,261900000,78.347826,8.0,1.0,27.304348,92175000.0,71.826087,0.0,0,0,0,0,1,0,1,0,0,0,0,11.0,E


In [74]:
world_cup = world_cup.drop(world_cup.index[11])



In [75]:
world_cup = world_cup.fillna(0)